In [1]:
# Устанавливаем OpenJDK
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# Закачиваем Spark
!wget -q http://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop2.7.tgz -O spark.tgz
# Распаковываем архив со Spark
!tar xf spark.tgz
# Устанавливаем пакет findspark для работы со Spark из Python
!pip install -q findspark
# Настраиваем переменные окружения для работы с Apache Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop2.7"
# Находим установку Spark
import findspark
findspark.init()
# Подключаем необходимые модули для работы со Spark из Python
from pyspark.sql import SparkSession
# Создаем сессию Spark на локальном компьютере
spark = SparkSession.builder.master("local[*]").getOrCreate()
!wget https://datahub.io/core/population/r/population.json -O population.json -q

##Задача 1 Модернизируйте заготовку заменив все участки <put your code here> на ваш код для того, чтобы:

вычислить прирост населения в каждой стране от года к году (воспользуйтесь функцией lag или lead)

вычислить средний прирост за период с 1990 по 2018 год

вычислить и вывести на экран страны с трендом на убыль населения с 1990 по 2018 годы

In [2]:
from pyspark.sql.functions import lag, col, coalesce, lit, avg

coalesce - Returns the first column that is not null.


avg - Aggregate function: returns the average of the values in a group.

In [32]:
df = spark.read.json("/content/population.json")

In [33]:
df.printSchema()

root
 |-- Country Code: string (nullable = true)
 |-- Country Name: string (nullable = true)
 |-- Value: double (nullable = true)
 |-- Year: long (nullable = true)



2018 год последний в этих данных

In [34]:
df1 = df.where('year > 2018')
df1.show()

+------------+------------+-----+----+
|Country Code|Country Name|Value|Year|
+------------+------------+-----+----+
+------------+------------+-----+----+



In [35]:
from pyspark.sql import Window

In [36]:
windowCountry = Window.partitionBy("Country Name").orderBy("Year")

Берем значения за период времени от 1990 года

In [37]:
df = df.where('year > 1990')

In [38]:
df = df.withColumn("cnt_country", coalesce(col('Value') - lag("Value").over(windowCountry), lit(0)))

In [39]:
df.show()

+------------+------------+-----------+----+-----------+
|Country Code|Country Name|      Value|Year|cnt_country|
+------------+------------+-----------+----+-----------+
|         AFG| Afghanistan|1.3299017E7|1991|        0.0|
|         AFG| Afghanistan|1.4485546E7|1992|  1186529.0|
|         AFG| Afghanistan|1.5816603E7|1993|  1331057.0|
|         AFG| Afghanistan|1.7075727E7|1994|  1259124.0|
|         AFG| Afghanistan|1.8110657E7|1995|  1034930.0|
|         AFG| Afghanistan|1.8853437E7|1996|   742780.0|
|         AFG| Afghanistan|1.9357126E7|1997|   503689.0|
|         AFG| Afghanistan|1.9737765E7|1998|   380639.0|
|         AFG| Afghanistan|2.0170844E7|1999|   433079.0|
|         AFG| Afghanistan|2.0779953E7|2000|   609109.0|
|         AFG| Afghanistan|2.1606988E7|2001|   827035.0|
|         AFG| Afghanistan| 2.260077E7|2002|   993782.0|
|         AFG| Afghanistan|2.3680871E7|2003|  1080101.0|
|         AFG| Afghanistan|2.4726684E7|2004|  1045813.0|
|         AFG| Afghanistan|2.56

Средний прирост по каждой стране

In [40]:
df = df.groupBy("Country Name").agg(avg("cnt_country").name("trend"))

In [41]:
df.show()

+-------------------+-------------------+
|       Country Name|              trend|
+-------------------+-------------------+
|        Afghanistan|  852620.3214285715|
|            Albania|           -14300.5|
|            Algeria|  565283.9285714285|
|     American Samoa| 242.14285714285714|
|            Andorra|             726.25|
|             Angola|  662887.8928571428|
|Antigua and Barbuda| 1175.6785714285713|
|         Arab World|         6823532.25|
|          Argentina|           407696.5|
|            Armenia|-19767.214285714286|
|              Aruba|            1472.25|
|          Australia| 274953.14285714284|
|            Austria|            38772.5|
|         Azerbaijan|  95314.28571428571|
|       Bahamas, The|  4451.142857142857|
|            Bahrain|  37845.67857142857|
|         Bangladesh| 1991318.2857142857|
|           Barbados|  883.4285714285714|
|            Belarus|-25375.035714285714|
|            Belgium|            51027.5|
+-------------------+-------------

Выведим страны с отрицательным трендом

In [42]:
df.where("trend < 0").show(20)

+--------------------+-------------------+
|        Country Name|              trend|
+--------------------+-------------------+
|             Albania|           -14300.5|
|             Armenia|-19767.214285714286|
|             Belarus|-25375.035714285714|
|Bosnia and Herzeg...|-37335.357142857145|
|            Bulgaria|-57404.642857142855|
|Central Europe an...|-284198.03571428574|
|             Croatia|-21470.678571428572|
|             Estonia|           -8547.75|
|             Georgia| -39619.67857142857|
|             Hungary|-21351.285714285714|
|              Kosovo| -1882.142857142857|
|              Latvia|-25835.964285714286|
|           Lithuania| -32235.39285714286|
|             Moldova|  -9624.07142857143|
|              Poland| -9694.392857142857|
|         Puerto Rico|-13105.607142857143|
|             Romania|-126250.35714285714|
|  Russian Federation|-148069.64285714287|
|              Serbia|           -21894.0|
|             Ukraine|-263498.35714285716|
+----------